<a href="https://colab.research.google.com/github/JulieWang417/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

### Load and clean the house-votes-84 data 

In [2]:
#upload local data to colap

from google.colab import files
uploaded = files.upload()

Saving house-votes-84.data to house-votes-84.data


In [7]:
import pandas as pd
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
df = pd.read_csv("house-votes-84.data",header=None,names=column_headers,na_values="?")
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [8]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [10]:
df.replace({'y':1,'n':0},inplace=True)
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [11]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [14]:
rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


### hypothesis testing: look at the physician-fee-freeze bill (two-sample T-test)

In [0]:
from scipy.stats import ttest_ind

In [17]:
print(rep['physician-fee-freeze'].mean())
dem['physician-fee-freeze'].mean()

0.9878787878787879


0.05405405405405406

In [18]:
ttest_ind(rep['physician-fee-freeze'],dem['physician-fee-freeze'],nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

*   T-statistic : 49.37 > 0
*   P_value : 1.994262314074344e-177 < 0.01

*   Since the P-value is much smaller than  0.01, I want to reject the null hypothesis, and T-statistic > 0 , I would say that republicans support more than democrats on the issue "physician-fee-freeze".






### hypothesis testing: look at the el-salvador-aidanti-satellite-ban bill (two-sample T-test)

In [20]:
print(rep['anti-satellite-ban'].mean())
print(dem['anti-satellite-ban'].mean())

0.24074074074074073
0.7722007722007722


In [21]:
ttest_ind(rep['anti-satellite-ban'],dem['anti-satellite-ban'],nan_policy='omit')

Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)

*   T-statistic : -12.53 < 0
*   P_value : 8.521033017443867e-31 < 0.01

*   Since the P-value is much smaller than  0.01, I want to reject the null hypothesis, and T-statistic < 0, that means democrats support more than  republicans on the issue "physician-fee-freeze".

### hypothesis testing: look at the water_project bill (two-sample T-test)

In [49]:
print(rep['water-project'].mean())
dem['water-project'].mean()

0.5067567567567568


0.502092050209205

In [50]:
ttest_ind(rep['water-project'],dem['water-project'],nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

*   T-statistic : 0.089 < 0
*   P_value : 0.93 < 0.01

*   Since the P-value is bigger than  0.1, we fail to reject the null hypothesis, and T-statistic = 0.089, really close to 0, i would say that there isn't a statistically significant difference between the way reps and dems voted on the issue "water-project".

### Stretch goals:Refactor your code into functions so it's easy to rerun with arbitrary variables

In [0]:
def ttest(x):
  return ttest_ind(rep[x],dem[x],nan_policy='omit')  

In [125]:
list_ttest=[]
for x in column_headers[1:]:
  list_ttest.append(ttest(x))
list_ttest

[Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18),
 Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485),
 Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77),
 Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177),
 Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68),
 Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20),
 Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31),
 Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54),
 Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47),
 Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066),
 Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15),
 Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64),
 Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709

In [126]:
a = list(column_headers[1:])
a

['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free',
 'south-africa']

In [0]:
ttest = {} 
for key in a: 
    for value in list_ttest: 
        ttest[key] = value 
        list_ttest.remove(value) 
        break 

In [128]:
ttest

{'aid-to-contras': Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54),
 'anti-satellite-ban': Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31),
 'budget': Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77),
 'crime': Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47),
 'duty-free': Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32),
 'education': Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64),
 'el-salvador-aid': Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68),
 'handicapped-infants': Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18),
 'immigration': Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066),
 'mx-missile': Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47),
 'physician-fee-freeze': Ttest_indResult(statistic=49.367081573

Conclusions:

*   on bills 'handicapped-infants', 'budget', 'anti-satellite-ban', 
 'aid-to-contras', 'mx-missile', 'synfuels', 'duty-free', 'south-africa', democrats support more than republicans
*   on bills 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 'immigration', 'education', 'right-to-sue', 'crime',
 , republicans support more than democrats

*   on bill "water-project", there may not be much of a difference between republics and democrats
